In [4]:
from astropy.cosmology import FlatLambdaCDM
from astropy.units import Quantity
from slsim.Lenses.lens_pop import LensPop
import numpy as np
import slsim.Sources as sources
import slsim.Deflectors as deflectors
import slsim.Pipelines as pipelines
from slsim.Sources.SourceCatalogues.QuasarCatalog.quasar_pop import QuasarRate
import pandas as pd

In [5]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Lens population --> Catalog


Here we demonstrate functionality of requesting a catalog from a population of lenses generated by SLSim.
We store properties of each lens object, and then accumulate the properties of each ``Lens`` in the population into a ``pandas.DataFrame``.
We store the DataFrame as a pickle file -- this means that we can store the Lens object itself in the DataFrame, reload the file, and still have all access to the ``Lens`` methods and functionalities.
Thus, we can store a ``LensPop`` sample, look at corner plots to understand the distribution of parameters in the ``LensPop``, and finally load in the ``Lens`` systems to further work on light curve simulation, image simulation etc.


### Loading in sky area

In [57]:
# define a cosmology
cosmo = FlatLambdaCDM(H0=70, Om0=0.3)


# define a sky area
galaxy_sky_area = Quantity(value=1, unit="deg2")
quasar_sky_area = Quantity(value=5, unit="deg2")
full_sky_area = Quantity(value=500, unit="deg2")


# define limits in the intrinsic deflector and source population (in addition
# to the skypy config
# file)

kwargs_deflector_cut = {"band": "i", "band_max": 30, "z_min": 0.01, "z_max": 5}
kwargs_source_cut = {"z_min": 0.001, "z_max": 5.0}

### Instantiate ``SkyPyPipeline`` to create sample of potential deflectors ``deflectors.AllLensGalaxies``

In [58]:
# generate galaxy population using skypy pipeline.
galaxy_simulation_pipeline = pipelines.SkyPyPipeline(
    skypy_config=None,
    sky_area=galaxy_sky_area,
    filters=None,
)

In [59]:
# Initiate deflector popiulation class
lens_galaxies = deflectors.AllLensGalaxies(
    red_galaxy_list=galaxy_simulation_pipeline.red_galaxies,
    blue_galaxy_list=galaxy_simulation_pipeline.blue_galaxies,
    kwargs_cut=kwargs_deflector_cut,
    kwargs_mass2light={},
    cosmo=cosmo,
    sky_area=galaxy_sky_area,
)

/Users/padmavenkatraman/Documents/StrongLensing/slsim/slsim/Deflectors/DeflectorPopulation/all_lens_galaxies.py:56: UserWarning: Angular size is converted to arcsec because provided input_catalog_type is skypy. If this is not correct, please refer to the documentation of the class you are using
  red_galaxy_list = catalog_with_angular_size_in_arcsec(
/Users/padmavenkatraman/Documents/StrongLensing/slsim/slsim/Deflectors/DeflectorPopulation/all_lens_galaxies.py:59: UserWarning: Angular size is converted to arcsec because provided input_catalog_type is skypy. If this is not correct, please refer to the documentation of the class you are using
  blue_galaxy_list = catalog_with_angular_size_in_arcsec(


### ``QuasarRate`` will generate N number of point sources over some sky area and redshift range.
``quasar_sample`` can be used to filter this sample of point sources according to observational magnitude cuts.

In [60]:
# Initiate QuasarRate class to generate quasar sample.
quasar_class = QuasarRate(
    cosmo=cosmo,
    sky_area=quasar_sky_area,
    noise=True,
    redshifts=np.linspace(0.001, 5.01, 100),  # these redshifts are provided
    # to match general slsim redshift range in skypy pipeline.
)
quasar_source = quasar_class.quasar_sample(m_min=15, m_max=30)

Once you've initialized a quasar sample with properties M_i and z, you can go on to assign some quasar variability properties, and generate a ``sources.PointSources`` class. This is the parallel to ``deflectors.AllLensGalaxies``.

In [70]:
# Prepare dictionary of agn variability kwargs
variable_agn_kwarg_dict = {
    "length_of_light_curve": 1000,
    "time_resolution": 1,
    "log_breakpoint_frequency": 1 / 20,
    "low_frequency_slope": 1,
    "high_frequency_slope": 3,
    "standard_deviation": 0.9,
}

kwargs_quasar = {
    "variability_model": "light_curve",
    "kwargs_variability": {"agn_lightcurve", "i", "r"},
    "agn_driving_variability_model": "bending_power_law",
    "agn_driving_kwargs_variability": variable_agn_kwarg_dict,
    "lightcurve_time": np.linspace(0, 1000, 5),
}
# Initiate source population class.
source_quasar = sources.PointSources(
    quasar_source,
    cosmo=cosmo,
    sky_area=quasar_sky_area,
    kwargs_cut=kwargs_source_cut,
    point_source_type="quasar",
    point_source_kwargs=kwargs_quasar,
)

### Generate lens population, with some optional cuts

In [71]:
# Initiate LensPop class to generate lensed quasar pop. We simulate lens pop in 500
# deg^2. If you want to simulate in larger sky, change sky area to your requirement.
quasar_lens_pop = LensPop(
    deflector_population=lens_galaxies,
    source_population=source_quasar,
    cosmo=cosmo,
    sky_area=full_sky_area,
)
kwargs_lens_cuts = {}

# drawing population
quasar_lens_population = quasar_lens_pop.draw_population(
    speed_factor=1000, kwargs_lens_cuts=kwargs_lens_cuts
)

In [ ]:
### view the first lensed system Lens object
chosen_lens = quasar_lens_population[0]

In [74]:
### Now we can see all the properties of a lens by calling lens.lens_to_df()
chosen_lens.lens_to_df()

,ID,deflector_mass_theta_E,deflector_mass_center_x,deflector_mass_center_y,deflector_mass_e1,deflector_mass_e2,deflector_mass_gamma1,deflector_mass_gamma2,deflector_mass_ra_0,deflector_mass_dec_0,...,micro_shear_0,micro_shear_1,micro_shear_angle_0,micro_shear_angle_1,point_source_arrival_time_0,point_source_arrival_time_1,external_shear,extended_lensed_mag,extended_unlensed_mag,extended_magnification
0,GAL-QSO-LENS_0.0297_0.0358,0.550143,0.029721,0.03583,0.309866,-0.043055,-0.016281,0.012508,0.0,0.0,...,0.205779,1.427453,0.625787,1.639357,-48.031075,6.108163,0.020531,NaN,NaN,NaN


### Population catalog generation

In [ ]:
# to get the catalog for a whole population, we can loop this
# this takes around a minute to run for ~ 3000 lenses
full_pop_df = pd.DataFrame()
for i, lens_obj in enumerate(quasar_lens_population):
    full_pop_df = lens_obj.lens_to_df(index=i, df=full_pop_df)

In [ ]:
### if you want to just look at the microlensing properties
### in general, you can call full_pop_df.columns to look at all available column names
micro_cols = [c for c in full_pop_df.columns if "micro" in c]
full_pop_df[micro_cols].head(5)

,micro_kappa_star_0,micro_kappa_star_1,micro_kappa_tot_0,micro_kappa_tot_1,micro_shear_0,micro_shear_1,micro_shear_angle_0,micro_shear_angle_1,micro_kappa_star_2,micro_kappa_star_3,micro_kappa_tot_2,micro_kappa_tot_3,micro_shear_2,micro_shear_3,micro_shear_angle_2,micro_shear_angle_3
0,0.067033,2.307260,0.246896,1.448278,0.205779,1.427453,0.625787,1.639357,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0.167285,1.775730,0.284132,1.641642,0.259102,1.614901,1.759644,1.552521,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0.044058,6.279247,0.218565,6.433125,0.218566,6.433125,2.456255,-2.952414,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0.380937,0.570430,0.405014,0.566273,0.405015,0.566272,3.063372,-1.701727,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0.226063,0.240627,0.219313,0.220635,0.217794,0.237121,3.093654,-2.550248,1.695999,2.420827,1.256755,1.635208,1.298218,1.676084,0.192419,0.214738


In [80]:
### if you want to look at quads
quads_df = full_pop_df[full_pop_df["num_ps_images"] == 4]  # total 182 rows
quads_df.head(5)

,ID,deflector_mass_theta_E,deflector_mass_center_x,deflector_mass_center_y,deflector_mass_e1,deflector_mass_e2,deflector_mass_gamma1,deflector_mass_gamma2,deflector_mass_ra_0,deflector_mass_dec_0,...,micro_kappa_star_2,micro_kappa_star_3,micro_kappa_tot_2,micro_kappa_tot_3,micro_shear_2,micro_shear_3,micro_shear_angle_2,micro_shear_angle_3,point_source_arrival_time_2,point_source_arrival_time_3
4,GAL-QSO-LENS_0.0053_0.0076,0.195055,0.005346,0.007598,-0.560726,-0.147078,0.023093,-0.025211,0.0,0.0,...,1.695999,2.420827,1.256755,1.635208,1.298218,1.676084,0.192419,0.214738,-3.288482,-1.948357
17,GAL-QSO-LENS_-0.0023_0.0541,0.343770,-0.002303,0.054069,0.013076,-0.304015,0.000000,-0.000000,0.0,0.0,...,0.319856,0.768203,0.565321,1.188358,0.565321,1.188357,1.243267,1.701113,-24.114119,-4.621832
24,GAL-QSO-LENS_-0.0164_-0.0162,0.243192,-0.016389,-0.016187,-0.467185,-0.354588,0.012041,-0.031719,0.0,0.0,...,0.681943,1.286553,0.822108,1.763339,0.843606,1.811125,1.101208,0.262343,-7.897475,-1.365587
25,GAL-QSO-LENS_-0.0021_-0.0533,1.242687,-0.002070,-0.053308,-0.348312,-0.354658,0.000000,-0.000000,0.0,0.0,...,0.286778,1.103812,0.649857,1.533273,0.649857,1.533274,1.492997,0.452922,-396.092275,-79.547227
27,GAL-QSO-LENS_-0.0293_0.0370,0.015207,-0.029345,0.036964,-0.251057,0.409905,0.017375,-0.011370,0.0,0.0,...,0.480241,0.583219,0.933123,1.314651,0.933886,1.318774,-1.209055,-0.838410,-0.034205,-0.020729


#### Save the population dataframe to a pickle file. This allows us to also save the Lens object and retain its functionality

In [ ]:
### Here, we explore additional functionality of saving the object to the df, and pickling the file
full_pop_df["lens_obj"] = None
for i, lens_obj in enumerate(quasar_lens_population):
    full_pop_df.loc[i, "lens_obj"] = lens_obj

In [82]:
full_pop_df.head()  # there are 3509 rows in total in this run
# recall that no observing cuts have been placed when querying lenses

,ID,deflector_mass_theta_E,deflector_mass_center_x,deflector_mass_center_y,deflector_mass_e1,deflector_mass_e2,deflector_mass_gamma1,deflector_mass_gamma2,deflector_mass_ra_0,deflector_mass_dec_0,...,micro_kappa_star_3,micro_kappa_tot_2,micro_kappa_tot_3,micro_shear_2,micro_shear_3,micro_shear_angle_2,micro_shear_angle_3,point_source_arrival_time_2,point_source_arrival_time_3,lens_obj
0,GAL-QSO-LENS_0.0297_0.0358,0.550143,0.029721,0.035830,0.309866,-0.043055,-0.016281,0.012508,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,<slsim.Lenses.lens.Lens object at 0x164339cd0>
1,GAL-QSO-LENS_0.0298_-0.0398,0.166071,0.029832,-0.039835,-0.065152,0.028096,-0.008076,-0.012822,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,<slsim.Lenses.lens.Lens object at 0x1644efad0>
2,GAL-QSO-LENS_-0.0253_0.0068,1.002484,-0.025280,0.006783,0.089747,0.304125,0.000000,0.000000,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,<slsim.Lenses.lens.Lens object at 0x164419210>
3,GAL-QSO-LENS_0.0006_-0.0477,0.167975,0.000649,-0.047688,-0.082892,0.037268,0.000000,-0.000000,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,<slsim.Lenses.lens.Lens object at 0x164904a90>
4,GAL-QSO-LENS_0.0053_0.0076,0.195055,0.005346,0.007598,-0.560726,-0.147078,0.023093,-0.025211,0.0,0.0,...,2.420827,1.256755,1.635208,1.298218,1.676084,0.192419,0.214738,-3.288482,-1.948357,<slsim.Lenses.lens.Lens object at 0x16461c950>


In [83]:
### save to pickle file
full_pop_df.to_pickle("lens_pop_data.pkl")

## Restart kernel and see if loaded in data is fully functional

In [ ]:
### uncomment and load this cell!

# from astropy.cosmology import FlatLambdaCDM
# from astropy.units import Quantity
# import slsim
# from slsim.Lenses.lens_pop import LensPop
# import numpy as np
# import slsim.Sources as sources
# import slsim.Deflectors as deflectors
# import slsim.Pipelines as pipelines
# from slsim.Sources.SourceCatalogues.QuasarCatalog.quasar_pop import QuasarRate
# import corner
# import matplotlib.pyplot as plt
# from scipy.stats import gaussian_kde
# import pandas as pd

# %load_ext autoreload
# %autoreload 2

In [ ]:
### load in pickle file
read_in_data = pd.read_pickle("lens_pop_data.pkl")
read_in_data.head()

,ID,deflector_mass_theta_E,deflector_mass_center_x,deflector_mass_center_y,deflector_mass_e1,deflector_mass_e2,deflector_mass_gamma1,deflector_mass_gamma2,deflector_mass_ra_0,deflector_mass_dec_0,...,micro_kappa_star_3,micro_kappa_tot_2,micro_kappa_tot_3,micro_shear_2,micro_shear_3,micro_shear_angle_2,micro_shear_angle_3,point_source_arrival_time_2,point_source_arrival_time_3,lens_obj
0,GAL-QSO-LENS_0.0297_0.0358,0.550143,0.029721,0.035830,0.309866,-0.043055,-0.016281,0.012508,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,<slsim.Lenses.lens.Lens object at 0x10770c110>
1,GAL-QSO-LENS_0.0298_-0.0398,0.166071,0.029832,-0.039835,-0.065152,0.028096,-0.008076,-0.012822,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,<slsim.Lenses.lens.Lens object at 0x17e1d7b50>
2,GAL-QSO-LENS_-0.0253_0.0068,1.002484,-0.025280,0.006783,0.089747,0.304125,0.000000,0.000000,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,<slsim.Lenses.lens.Lens object at 0x17e2f7450>
3,GAL-QSO-LENS_0.0006_-0.0477,0.167975,0.000649,-0.047688,-0.082892,0.037268,0.000000,-0.000000,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,<slsim.Lenses.lens.Lens object at 0x17e30c9d0>
4,GAL-QSO-LENS_0.0053_0.0076,0.195055,0.005346,0.007598,-0.560726,-0.147078,0.023093,-0.025211,0.0,0.0,...,2.420827,1.256755,1.635208,1.298218,1.676084,0.192419,0.214738,-3.288482,-1.948357,<slsim.Lenses.lens.Lens object at 0x17e30d890>


In [ ]:
# make sure you've imported all the SLSim methods 2 cells above
# load in lens object and make sure method works.
# here, we can see that the loaded in lens object is still able to produce magnitude at different times.
first_lens = read_in_data.loc[0, "lens_obj"]
first_lens.point_source_magnitude(
    band="i", lensed=True, time=np.linspace(0, 1000, 1000)
)

[array([[29.08325616, 29.0832989 , 29.08334164, ..., 29.12124228,
         29.12127416, 29.12130603],
        [30.4433048 , 30.4433048 , 30.4433048 , ..., 30.47956698,
         30.47959886, 30.47963073]])]